Experiment Type 1 : Regression , Dirichelet; alpha=1  

In [1]:
import sys
sys.path.append("../..")
from flwr_datasets.partitioner import DirichletPartitioner
import torch.nn as nn
from torch.optim import SGD
from moxi import create_experiment

/home/sgtec/miniconda3/envs/moxifl/lib/python3.11/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


In [2]:
config = {
  "network_name": "federated_cnn_experiment_1",
  "network_type": "centralised",
  "model_type": "parametric",
  "ml_framework": "pytorch",
  "comms": "async",
  "federated_rounds": 5,
  "network_size": 5,
  "metrics": ["mean_perfomance","convergence"],
  "logger": "mlflow",
  "node_base_config":{
      "model":None,
      "learning_rate": 1e-3,
      "optimizer": SGD,
      "train_data":None,
      "val_data":None,
      "criterion":nn.CrossEntropyLoss,
      "random_sampling":False,
      "n_epochs": 5,
      "batch_size": 16
  },
  "experiment_config":{
      "task": "image_classification",
      "partitioner": DirichletPartitioner,
      "alpha": 1,
      "num_worker": 2,
      "max_samples": 1000
  },
  "adjcency_matrix": None,
  "model": None }

In [3]:
class CNNModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(128 * 4 * 4, 512)
        self.fc2 = nn.Linear(512, 10)
        self.maxpool = nn.MaxPool2d(2, 2)
        self.activation = nn.ReLU()
    def forward(self, x):
        x = self.activation(self.maxpool(self.conv1(x)))
        x = self.activation(self.maxpool(self.conv2(x)))
        x = self.activation(self.maxpool(self.conv3(x)))
        x = x.view(-1, 128 * 4 * 4)
        x = self.activation(self.fc1(x))
        x = self.fc2(x)
        return x


In [4]:
model = CNNModel()

In [5]:
flnw = create_experiment(config, model)


Loading federated CIFAR-10 dataset: uoft-cs/cifar10


In [6]:
async def execute_experiment(flnw, num_rounds:int, epochs_per_round:int, experiment_name:str):
    await flnw.train(num_rounds= num_rounds, epochs_per_round=epochs_per_round, experiment_name=experiment_name)
    print("Experiment completed.")

In [7]:
import nest_asyncio
nest_asyncio.apply()
await execute_experiment(flnw, num_rounds=1, epochs_per_round=1, experiment_name="federated_mnist_experiment")

2025/09/09 00:31:11 INFO mlflow.tracking.fluent: Experiment with name 'federated_mnist_experiment' does not exist. Creating a new experiment.


Training Complete!
Experiment completed.
